<a href="https://colab.research.google.com/github/HasikaR/Audio-Denoising-Deep-Learning/blob/main/Testing(h).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json
import numpy
import os

In [ ]:
with zipfile.ZipFile('/content/drive/MyDrive/s6.zip', 'r') as zip_ref:
    zip_ref.extractall('/tmp3')

In [ ]:
with zipfile.ZipFile('/content/drive/MyDrive/c6.zip', 'r') as zip_ref:
    zip_ref.extractall('/tmp4')

In [ ]:
import librosa
import numpy as np

In [ ]:
#test samples
#load clean signal filenames
files_path3='/tmp4/c6'
tc_files=librosa.util.find_files(files_path3, ext=['wav'])

#load synthetic signal filenames
files_path4='/tmp3/s6'
ts_files=librosa.util.find_files(files_path4, ext=['wav'])

#create a multicolumn object with synthetic and clean signal names
tc1_arr=np.array(tc_files)
ts1_arr=np.array(ts_files)
t_samples= np.column_stack((tc1_arr,ts1_arr))
test_samples=np.array(t_samples)
#test_samples=ts1_arr

In [ ]:
print(len(test_samples))

36


In [ ]:
y, sr = librosa.load('/tmp3/s6/ss2221_cs75_kd1.wav', sr =None)

In [ ]:
print(y)
print(y.shape)

[-0.00018311 -0.00064087 -0.00012207 ... -0.00479126  0.00082397
  0.        ]
(576000,)


In [ ]:
print(ts1_arr)

['/tmp3/s6/ss2221_cs75_kd1.wav' '/tmp3/s6/ss2222_cs75_kd2.wav'
 '/tmp3/s6/ss2223_cs75_kd3.wav' '/tmp3/s6/ss2224_cs75_kd4.wav'
 '/tmp3/s6/ss2225_cs75_kd5.wav' '/tmp3/s6/ss2226_cs75_kd6.wav'
 '/tmp3/s6/ss2227_cs75_kd7.wav' '/tmp3/s6/ss2228_cs75_kd8.wav'
 '/tmp3/s6/ss3481_cs42_tci1.wav' '/tmp3/s6/ss3482_cs42_tci2.wav'
 '/tmp3/s6/ss3483_cs42_tci3.wav' '/tmp3/s6/ss3484_cs42_tci4.wav'
 '/tmp3/s6/ss3485_cs42_tci5.wav' '/tmp3/s6/ss3486_cs42_tci6.wav'
 '/tmp3/s6/ss3487_cs42_tci7.wav' '/tmp3/s6/ss4531_cs2_vc1.wav'
 '/tmp3/s6/ss4532_cs2_vc2.wav' '/tmp3/s6/ss4533_cs2_vc3.wav'
 '/tmp3/s6/ss4534_cs2_vc4.wav' '/tmp3/s6/ss4535_cs2_vc5.wav'
 '/tmp3/s6/ss4536_cs2_vc6.wav' '/tmp3/s6/ss4537_cs2_vc7.wav'
 '/tmp3/s6/ss7351_cs21_acf1.wav' '/tmp3/s6/ss7352_cs21_acf2.wav'
 '/tmp3/s6/ss7353_cs21_acf3.wav' '/tmp3/s6/ss7354_cs21_acf4.wav'
 '/tmp3/s6/ss7355_cs21_acf5.wav' '/tmp3/s6/ss7356_cs21_acf6.wav'
 '/tmp3/s6/ss7357_cs21_acf7.wav' '/tmp3/s6/ss9811_cs28_r1.wav'
 '/tmp3/s6/ss9812_cs28_r2.wav' '/tmp3/s6/ss9813_c

In [ ]:
#install pydub for audio slicing
!pip install pydub

In [ ]:
from pydub import AudioSegment
from pydub.utils import make_chunks
sig_slice = []
def audio_slicer(file_name):
    myaudio = AudioSegment.from_file(file_name , "wav") 
    chunk_length_ms = 12000 # pydub calculates in millisec
    chunks = make_chunks(myaudio, chunk_length_ms) #Make chunks of one sec
        
    #Export all of the individual chunks as wav files
    for i, chunk in enumerate(chunks):
        chunk_name = "chunk{0}.wav".format(i)
        chunk.export(chunk_name, format="wav")
        data,sr = librosa.load(chunk_name, sr=None)
        sig_slice.append(data)
    return sig_slice

In [ ]:
def generator(samples, batch_size):
  num_samples = len(samples)
  while True:
      for offset in range(0, num_samples, batch_size):
            # Get the samples you'll use in this batch
            batch_samples = samples[offset:offset+batch_size]
 
            # Initialise X_train and y_train arrays for this batch
            X_train = []
            y_train = []
            count = 0

            # For each example
            for _ in batch_samples:
                # Load image (X)
                i=0;
                count += 12
                syn_filename = samples[i][0]
                sliced_syn_signal = audio_slicer(syn_filename)
                        
                X_train.append(sliced_syn_signal)
                # Read label (y)
               
                clean_filename = samples[i][1]
                sliced_cln_signal = audio_slicer(syn_filename)
                y_train.append(sliced_cln_signal)
                print(y_train)
                
                if count >= batch_size:
                    # Make sure they're numpy arrays (as opposed to lists)
                    X_train = np.asarray(X_train)
                    #X_train = X_train.reshape(1,48000,1)
                    
                    y_train = np.array(y_train)
                    #y_train = y_train.reshape(1,48000,1)
                    #print(X_train.shape)
                    #print(y_train.shape)
                    # The generator-y part: yield the next training batch            
                    yield X_train, y_train

                    #clear data 
                    X_train = []
                    y_train = []
                    count = 0

In [ ]:
test_generator = generator(test_samples, batch_size=36)

In [ ]:
json_file = open('/content/drive/MyDrive/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("/content/drive/MyDrive/model.h5")
print("Loaded model from disk")
 


Loaded model from disk


In [ ]:
loaded_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, None, 16)          33869888  
_________________________________________________________________
time_distributed (TimeDistri (None, None, 529200)      8996400   
Total params: 42,866,288
Trainable params: 42,866,288
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#loaded_model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
import keras

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import TimeDistributed
loaded_model.compile(loss='mean_squared_error', optimizer='adam')
mycall = [EarlyStopping(monitor='loss', min_delta=0.001, patience=1, mode='min', verbose=1),ModelCheckpoint(filepath="/content/drive/MyDrive/weights.hdf5", monitor = 'loss', verbose=1, save_best_only=True,mode='min', save_freq = 1)]

In [ ]:
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.history = {'loss':[],'val_loss':[]}

    def on_batch_end(self, batch, logs={}):
        self.history['loss'].append(logs.get('loss'))

    def on_epoch_end(self, epoch, logs={}):
        self.history['val_loss'].append(logs.get('val_loss'))

In [ ]:
class LossAndErrorPrintingCallback(keras.callbacks.Callback):
    def on_train_batch_end(self, batch, logs=None):
        print("For batch {}, loss is {:7.2f}.".format(batch, logs["loss"]))

    def on_test_batch_end(self, batch, logs=None):
        print("For batch {}, loss is {:7.2f}.".format(batch, logs["loss"]))

    def on_epoch_end(self, epoch, logs=None):
        print(
            "The average loss for epoch {} is {:7.2f} "
            "and mean absolute error is {:7.2f}.".format(
                epoch, logs["loss"], logs["mean_absolute_error"]
            )
        )

In [ ]:
import tensorflow as tf
import gc

class MyCustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()
        keras.backend.clear_session()

In [ ]:
#predictions
ptest =loaded_model.predict(test_generator, steps = len(test_samples)//36,callbacks=[LossHistory(),MyCustomCallback()], workers=1,verbose=1)
print(ptest)


[[array([-3.0517578e-05, -1.2207031e-04,  0.0000000e+00, ...,
       -1.2207031e-03,  9.1552734e-05,  5.4931641e-04], dtype=float32), array([-3.0517578e-05, -1.2207031e-04,  0.0000000e+00, ...,
       -1.2207031e-03,  9.1552734e-05,  5.4931641e-04], dtype=float32)]]
[[array([-3.0517578e-05, -1.2207031e-04,  0.0000000e+00, ...,
       -1.2207031e-03,  9.1552734e-05,  5.4931641e-04], dtype=float32), array([-3.0517578e-05, -1.2207031e-04,  0.0000000e+00, ...,
       -1.2207031e-03,  9.1552734e-05,  5.4931641e-04], dtype=float32), array([-3.0517578e-05, -1.2207031e-04,  0.0000000e+00, ...,
       -1.2207031e-03,  9.1552734e-05,  5.4931641e-04], dtype=float32), array([-3.0517578e-05, -1.2207031e-04,  0.0000000e+00, ...,
       -1.2207031e-03,  9.1552734e-05,  5.4931641e-04], dtype=float32)], [array([-3.0517578e-05, -1.2207031e-04,  0.0000000e+00, ...,
       -1.2207031e-03,  9.1552734e-05,  5.4931641e-04], dtype=float32), array([-3.0517578e-05, -1.2207031e-04,  0.0000000e+00, ...,
       -1

In [ ]:
print(type(ptest))

<class 'numpy.ndarray'>


In [ ]:
history = LossHistory()

In [ ]:
print(ptest.shape)

(3, 72, 44100)


In [ ]:
#%store -r numpy_loss_history

no stored variable numpy_loss_history


In [ ]:

#convert numpy array to audio file
from pydub import AudioSegment as export
import numpy as np
import librosa
import soundfile as sf
from scipy.io.wavfile import write

from pydub.playback import play
from IPython.display import Audio
xx=[]

for j, sig in enumerate(ptest[0]):
    pt1 = "sig{0}.wav".format(j)
    print(sig[0])
    xx.append(sig[0])
    fs = 44100
    #sd.play(sig, fs)

    #sig.export('/content/drive/MyDrive/lstm_output/predictoutputpt1'+pt1, format="wav")
    write('/content/drive/MyDrive/lstm_output3/predictoutputpt2'+pt1,fs,sig)
    

-1.8788967e-05
-0.00010847943
-0.00019168766
-0.0002670521
-0.00033406305
-0.0003928212


In [ ]:
print(xx)

[-0.009047418, -0.009631066, -0.00913505, -0.009571094, -0.007971369, -0.00877447, -0.008882075, -0.00957768, -0.009071277, -0.009532358, -0.0079289125, -0.008763558, -0.008869739, -0.009574869, -0.009059534, -0.009519059, -0.007913397, -0.00875614, -0.008861436, -0.009568427, -0.009052487, -0.009511639, -0.0079064965, -0.008751753, -0.008857286, -0.0095650675, -0.009049422, -0.009508866, -0.007904173, -0.008750099, -0.008855854, -0.009563948, -0.009048439, -0.009508102, -0.007903577, -0.008749636]


In [ ]:

from pydub import AudioSegment

sound1 = AudioSegment.from_file("/content/drive/MyDrive/lstm_output3/predictoutputpt2sig0.wav", format="wav")

# sound1 6 dB louder
louder = sound1 + 6

# simple export
louder.export(out_f = "/content/drive/MyDrive/lstm_output/output19.wav", format="wav")


<_io.BufferedRandom name='/content/drive/MyDrive/lstm_output/output19.wav'>